In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

In [2]:
df=pd.read_excel('C:/Users/sicon/Downloads/Retail Case Files/RetailerPromotionStrategy_data.xlsx',header=0)

In [172]:
df.head()

,StoreID,BasketID,Line,Sales,Count,Date,SKU
0,X29,11-7270-00007-00057,36,850.0,1.0,2012-03-11,A21773
1,X29,11-7270-00007-00057,39,1500.0,1.0,2012-03-11,A18009
2,X29,11-7270-00007-00057,40,1500.0,1.0,2012-03-11,A25592
3,X29,11-7270-00007-00057,41,1600.0,1.0,2012-03-11,A1449
4,X29,11-7270-00007-00057,42,1600.0,1.0,2012-03-11,A1449


**1. For their top 5 selling products, on average, do people purchase more items when given a discount?**

In [345]:
# Find out baskets that are mainly returning product bought in previous circle
df1=df.groupby(by=['BasketID'])['Count'].sum()
df1=pd.DataFrame(df1)
return_item=df1[df1['Count']<0].index
df2=df[-df["BasketID"].isin(return_item)]

# Top five selling product accoding to number of item sold
Top_item=df2.groupby(by=['SKU'])['Count'].sum()
Top_5=Top_item.sort_values(ascending=False)[:5]
Top_5.index


Index([u'A10809', u'A3664', u'A21771', u'A2854', u'A25312'], dtype='object', name=u'SKU')

In [174]:
Top_5

SKU
A10809    35579.0
A3664     30904.0
A21771    23433.0
A2854     15574.0
A25312    13012.0
Name: Count, dtype: float64

In [190]:
# Find promotion index
index2=df2[(df2['SKU'].isin(Top_5.index))&(df2['Count']==0)].index
# Find number of goods bought during promotion

temp=df2.loc[index2-1]
Top_5_onsale=temp[temp['SKU'].isin(Top_5.index)]

# Number of promotion not used immediately, total 34, can be ignored
later=len(temp[-temp['SKU'].isin(Top_5.index)].index)

Top_5_onsale_sum=Top_5_onsale.groupby(by=['SKU'])['Count'].sum()
Top_5_not_onsale=Top_5-Top_5_onsale_sum
#len(Top_5_onsale['SKU'].unique())
Top_5_onsale_sum

SKU
A10809    34811.0
A21771        1.0
A25312     8396.0
A3664         4.0
Name: Count, dtype: float64

In [191]:

Top_5_onsale_sum['A2854']=0 
Top_5_onsale_sum['A21771']=0
Top_5_onsale_sum['A3664']=0
Top_5_onsale_sum

SKU
A10809    34811.0
A21771        0.0
A25312     8396.0
A3664         0.0
A2854         0.0
Name: Count, dtype: float64

In [192]:
Top_5_not_onsale=Top_5-Top_5_onsale_sum
Top_5_not_onsale

SKU
A10809      768.0
A21771    23433.0
A25312     4616.0
A2854     15574.0
A3664     30904.0
Name: Count, dtype: float64

In [196]:
# Calculate average
count_onsale=df2[(df2['SKU'].isin(Top_5.index))&(df2['Count']==0)].groupby(by=['SKU'])['Count'].count()
percentage=[]
for i in ['A10809','A25312']:
    percentage.append(Top_5_onsale_sum[i]/count_onsale[i])


#count_not_onsale=df2[(df2['SKU'].isin(['A21771','A2854']))&(df2['Count']>0)].groupby(by=['SKU'])['Count'].count()

count_not_onsale2=df2[(df2['SKU'].isin(['A10809','A25312']))&(df2['Count']>0)].groupby(by=['SKU'])['Count'].count()

#count_not_onsale2    
percentage1=[]

for i in ['A10809','A25312']:
    percentage1.append(Top_5_not_onsale[i]/(count_not_onsale2[i]-count_onsale[i]))
        
    
   

In [197]:
percentage

[12.630986937590711, 1.0264058679706602]

In [198]:
percentage1 # Promotion increases average amount people buy

[3.0236220472440944, 1.0036964557512502]

2. For their top 5 selling products, which product has the highest demand elasticity?

In [333]:
#find onsasle index
df21=df2[(df2['SKU']=='A10809')&(df2['Count']==0)]
index3=df21.index
#find onsale trasaction
df3=df2.loc[index3-1]
df30=df3[df3['SKU']=="A10809"]

#'A10809' onsale transaction
df4=df21.merge(df30.rename(columns={"Sales":"Sales0","Count":"Count0"}),on=['BasketID'])


#df5=df2.loc[df4.index+1]

df4['Price']=(df4.reset_index()['Sales']+df4.reset_index()['Sales0'])/df4.reset_index()['Count0']

#non_onsale transaction
k=df3.index
df6=df2.drop(k)


In [271]:
df7=df6[(df6['SKU']=='A10809')&(df6['Count']>0)]
df7['Price']=df7['Sales']/df7['Count']
df7['Price'].unique()

C:\Users\sicon\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


array([ 500.])

In [337]:
by_store=df7.groupby(by=['StoreID'])['Count'].sum()
by_store_onsale=df4.groupby(by=['StoreID_x'])['Count0'].sum()
by_store

StoreID
X03     23.0
X11     10.0
X14     13.0
X17     94.0
X21      8.0
X23     10.0
X24     10.0
X27     21.0
X38     10.0
X43     30.0
X44     10.0
X45     22.0
X49     10.0
X53     11.0
X55     49.0
X59     77.0
X60     72.0
X63     30.0
X65     27.0
X72    139.0
X78    212.0
X85     23.0
Name: Count, dtype: float64

In [339]:
count1=df7.groupby(by=['StoreID'])['Count'].count()
count2=df4.groupby(by=['StoreID_x'])['Count'].count()
demand_not_onsale=by_store/count1
demand_onsale=by_store_onsale/count2
elasticity=((demand_onsale[by_store.index]-demand_not_onsale)/demand_not_onsale)/0.3
elasticity

StoreID
X03    15.072464
X11     1.647059
X14     8.026125
X17     5.522775
X21    11.862745
X23     1.166667
X24     2.666667
X27     2.532468
X38     0.609553
X43     1.643519
X44     2.076923
X45    17.445887
X49     4.933333
X53     4.181818
X55     8.723860
X59    21.847432
X60    -1.274462
X63     4.418605
X65     4.784580
X72     8.637534
X78    16.216518
X85     3.109857
dtype: float64

3. For this product (identified in question #2) or the top selling product, does the number of days on promotion impact the effectiveness of the promotion (i.e. how many units people buy)?

In [362]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
    
print_full(df8)

Date_x      StoreID_x
2012-03-11  X17          12.876543
            X27          11.209302
            X29          11.785714
            X38          11.640187
            X43          13.073684
            X55          10.425926
            X60          10.778894
            X65          10.987654
            X71          11.038462
            X72           9.984848
            X78          11.463768
            X85          11.114504
            X97          11.094595
            X99          11.250000
2012-03-12  X01          10.857143
            X02          11.714286
            X03          12.700000
            X05          10.000000
            X06          30.000000
            X07          15.185185
            X09          11.176471
            X10          10.000000
            X11          14.941176
            X12          10.000000
            X13          14.181818
            X14          11.075472
            X15          16.125000
            X16          10.00000

In [360]:
df8=df4.groupby(by=['Date_x','StoreID_x'])['Count0'].sum()/df4.groupby(by=['Date_x','StoreID_x'])['Count0'].count()
print (df8)



Date_x      StoreID_x
2012-03-11  X17          12.876543
            X27          11.209302
            X29          11.785714
            X38          11.640187
            X43          13.073684
            X55          10.425926
            X60          10.778894
            X65          10.987654
            X71          11.038462
            X72           9.984848
            X78          11.463768
            X85          11.114504
            X97          11.094595
            X99          11.250000
2012-03-12  X01          10.857143
            X02          11.714286
            X03          12.700000
            X05          10.000000
            X06          30.000000
            X07          15.185185
            X09          11.176471
            X10          10.000000
            X11          14.941176
            X12          10.000000
            X13          14.181818
            X14          11.075472
            X15          16.125000
            X16          10.00000

In [ ]:
df8.loc[(slice(),['x17,x']]